In [1]:
from dask.distributed import Client
dask_client = Client()
dask_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54126,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:54137,Total threads: 2
Dashboard: http://127.0.0.1:54141/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:54129,


In [2]:
import getpass

In [3]:
cid = getpass.getpass(prompt="client_id for box")

client_id for box ········


In [4]:
cid

'l8viwaotvjn2b5aqeyrm18az8a5gy9zx'

In [5]:
cs = getpass.getpass(prompt="client_secret for box")

client_secret for box ········


In [6]:
cs

'Wo58BWDE2WYsV3ps1vpQg0C3KjLAasQk'

In [8]:
atk = getpass.getpass(prompt="access token for box")

access token for box ········


In [9]:
atk

'1tIVlL16aEw2mfqG7DoNKuUGHNtmg0FW'

In [10]:
from boxsdk import OAuth2, Client

In [11]:
auth = OAuth2(client_id=cid, client_secret=cs, access_token=atk)

In [12]:
client = Client(auth)

In [13]:
client.auth

In [14]:
file_id = '1278387018073'
file_content = client.file(file_id).content()

In [41]:
import io
file_id = '1281263195847'
file_content = client.file(file_id).content()
downloaded_data =  io.BytesIO(file_content)

In [42]:
downloaded_data.getbuffer()

In [43]:
# Write the stuff
with open("downloaded_file.csv", "wb") as f:
    f.write(downloaded_data.getbuffer())

In [57]:
import dask.dataframe as dd
from dask.dataframe import to_datetime
ddf = dd.read_csv("downloaded_file.csv", parse_dates=True, date_format='%Y-%m-%d %H:%M:%S')
ddf

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,string,string,bool,int64,int64,int64,bool,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,bool,bool,string,string,string,string,string,string,string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [58]:
ddf.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [69]:
cols_closed = ["number", "resolved_at", "incident_state", "assignment_group"]
ddf = ddf[cols_closed].copy()

In [70]:
ddf.head()

,number,resolved_at,incident_state,assignment_group
3,INC0000045,2016-02-29 11:29:00,Closed,Group 56
12,INC0000047,2016-03-01 09:52:00,Closed,Group 24
19,INC0000057,2016-03-01 02:55:00,Closed,Group 70
23,INC0000060,2016-03-02 12:06:00,Closed,Group 25
31,INC0000062,2016-02-29 15:51:00,Closed,Group 23


In [71]:
selection_closed = (ddf["incident_state"] == "Closed" ) & ~(ddf["resolved_at"] == "?" ) \
& ~(ddf["assignment_group"] == "?" )
ddf = ddf[selection_closed]

In [72]:
from dask.dataframe import to_datetime
ddf["resolved_at"] = to_datetime(ddf["resolved_at"])

In [73]:
ddf_Q2 = ddf[ddf["resolved_at"].dt.quarter == 2]

In [75]:
ag_groups = ddf.groupby("assignment_group")
ag_groups["number"].count()

assignment_group
Group 10    346
Group 12    123
Group 13     36
Group 14     13
Group 15     58
           ... 
Group 76    181
Group 77      1
Group 8       2
Group 80      1
Group 9      96
Name: number, Length: 70, dtype: int64

In [82]:
import numpy as np
import h5py